The following code snippet imports the TensorRT module from tensorflow.contrib libraries. Be sure to use it on a linux machine for best compatibility

In [1]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 512kB 37.3MB/s 
     |████████████████████████████████| 3.8MB 39.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=c255e72d6b216e0d9ce8ffcc89e752cdc1a7843c5ba494d7a3c2a8955421e253
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninst

In [2]:
import tensorflow.contrib.tensorrt

The following code mounts google drive, which is necessary in case you wish to use a model from google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Freeze Graph
Incase you're using .h5 model, you need to freeze it before you can apply tensorRT optimizations. Locate your .h5 file and run the code cell below.

In [5]:
import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model


# Clear any previous session.
tf.keras.backend.clear_session()

save_pb_dir = './model'
model_fname = './drive/My Drive/alexnet_final_lr001.h5'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

['conv2d_input'] ['activation_8/Softmax']
INFO:tensorflow:Froze 50 variables.
INFO:tensorflow:Converted 50 variables to const ops.


# TensorRT optimizations
The code cell below applies the TensorRT optimizations. You can try different quantization levels by changing the `precision_mode = 'FP16'` .

In [ ]:
from tensorflow.contrib import tensorrt

trt_graph = tensorrt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

# Save the graph
Run this code cell to save the TensorRT optimized model to your desired location on google drive.

In [ ]:
graph_io.write_graph(trt_graph, "./model/",
                     "trt_alexnet.pb", as_text=False)

'./model/trt_graph16.pb'

# References
1. Dlology, "How to run Keras model on Jetson Nano", . [Online]. Available https://www.dlology.com/blog/how-to-run-keras-model-on-jetson-nano/